In [48]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
import datetime
from dataclasses import dataclass,asdict

In [2]:
CLEF_API = "API_KEY"

In [3]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [4]:
def gare_page(page_num):
    '''Requete une liste de gare à une page spécifique'''
    reponse = requests.get(f"https://api.sncf.com/v1/coverage/sncf/stop_areas?start_page={page_num}", auth=(api_key,""))
    return reponse.json()

In [5]:
def ecrire_json(data,nom_fichier):
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [6]:
def nombre_pages(nb_gares,nb_gares_par_page):
    '''Calcul le nombre de pages de la requete'''
    nb_pages = nb_gares // nb_gares_par_page
    # On calcul le modulo (reste de la division)
    modulo = nb_gares % nb_gares_par_page
    # Si le reste de la division est différent de 0, alors on ajoute une dernière page
    if modulo != 0:
        nb_pages += 1
    return nb_pages

In [52]:
@dataclass
class DataClassGare:
    id : str | None
    nom : str | None
    timezone : str | None
    label : str | None
    longitude : str | None
    latitude : str | None
    id_region_ad : str | None
    nom_region_ad : str | None
    zip_code : str | None
    insee_region_ad : str | None


In [63]:
def extraire_donnees(liste_gares,clef):
    '''Cette fonction permet d'extraire une donnée dans un dictionnaire
    Si la donnée est de profondeur 1 la clef est un string
    Si la donnée est de profondeur > 1 la clef est une liste '''

    try:
        # Extraire une donnée de profondeur 1
        if type(clef) is str:
            data = liste_gares[clef]
        # Extraire une donnée de profondeur > 1
        else:
            data = liste_gares
            # Gestion du cas particulier de la gestion administrative
            if clef[0] == "administrative_regions":
                for x in clef[1:]:
                    data = liste_gares["administrative_regions"][0][x]
            # autres cas
            else:
                for x in clef:
                    data = data[x]
        return data
    
    except KeyError:
        return None

In [56]:
def recuperer_data(donnees):
    '''extrait des données et les retourne sous forme de dictionnaire'''
    # Je recupère toutes les donnéee de toutes les gares.

    gare = DataClassGare(
        id = extraire_donnees(donnees,"id"),
        nom = extraire_donnees(donnees,"name"),
        timezone=extraire_donnees(donnees,"timezone"),
        label=extraire_donnees(donnees,"label"),
        longitude=extraire_donnees(donnees,["coord","lon"]),
        id_region_ad=extraire_donnees(donnees,["coord","lon"]),
        nom_region_ad=extraire_donnees(donnees,["administrative_regions","name"]),
        zip_code=extraire_donnees(donnees,["administrative_regions","zip_code"]),
        insee_region_ad=extraire_donnees(donnees,["administrative_regions","insee"])
    )

    return asdict(gare)

In [19]:
premiere_page = gare_page(0)
nb_gares = premiere_page["pagination"]["total_result"]
gares_par_page = premiere_page["pagination"]["items_per_page"]
nb_pages = nombre_pages(nb_gares,gares_par_page)
print(f"Il y a {gares_par_page} gares par page, et {nb_gares} gares au total. Il y a donc {nb_pages} au total")
gares = premiere_page["stop_areas"]

Il y a 25 gares par page, et 4494 gares au total. Il y a donc 180 au total


In [57]:
gare = recuperer_data(gares[0])
print(gare)

KeyError: 'administrative_regions'

In [32]:
for page in range(nb_pages):
    requete = gare_page(page)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
